<a href="https://colab.research.google.com/github/pushkershukla/adv_fairness/blob/main/Training_Face_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torchattacks

In [ ]:
# all imports
!pip install logger
import os
import json
import numpy as np
import PIL.Image as Image
import logger
import copy
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch import optim 
from torchvision import datasets, transforms, models
import torch.utils.data as data
import zipfile
import pandas as pd
import torchvision
import time
import pandas
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

# reproducibility
seed = 42
# SYSTEM
np.random.seed(seed)
#CUDA
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed)
# CUDNN
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# version
print("PyTorch version {}".format(torch.__version__))
print("GPU-enabled installation? {}".format(torch.cuda.is_available()))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1780 sha256=2d4202854bc1e8eadc3c502983fb868cab921c38bdd2f595d3943c0924f80b18
  Stored in directory: /root/.cache/pip/wheels/fb/19/7b/09fc73f7503166eaf7f31b4aa0095b7f78af2ec0898e1f8312
Successfully built logger
[01/May/2023 05:46:20] INFO - NumExpr defaulting to 2 threads.
PyTorch version 2.0.0+cu118
GPU-enabled installation? True


Check alloted GPU/CUDA

In [ ]:
!!nvidia-smi
# preferred: Tesla V100-SXM2, CUDA v11.2

['Tue Mar 28 21:15:58 2023       ',
 '+-----------------------------------------------------------------------------+',
 '| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |',
 '|-------------------------------+----------------------+----------------------+',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
 '|                               |                      |               MIG M. |',
 '|===============================+======================+======================|',
 '|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |',
 '| N/A   61C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |',
 '|                               |                      |                  N/A |',
 '+-------------------------------+----------------------+----------------------+',
 '                                      

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Download CelebA dataset

---

In [ ]:
# mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#!mkdir -p /content/gdrive/MyDrive/downloads && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip -P /content/gdrive/MyDrive/downloads 

In [ ]:
dataset_dir='/content/gdrive/MyDrive/downloads/celeba/celeba.zip'

with zipfile.ZipFile(dataset_dir,"r") as zip_ref:
  zip_ref.extractall("faces/")

with zipfile.ZipFile("faces/celeba/img_align_celeba.zip","r") as zip_ref:
  zip_ref.extractall("faces/celeba")

In [ ]:
!ls faces/celeba
#!ls 

identity_CelebA.txt   list_attr_celeba.txt     list_landmarks_align_celeba.txt
img_align_celeba      list_bbox_celeba.txt     list_landmarks_celeba.txt
img_align_celeba.zip  list_eval_partition.txt


In [ ]:
# image_path="faces"
filepath="faces/celeba/list_attr_celeba.txt"
data_root='faces/celeba'
dataset_folder = f'{data_root}/img_align_celeba'
print(dataset_folder)
'''

celeba_train_dataset = torchvision.datasets.CelebA(
     image_path, split='train',
     target_type='attr', download=False
 )
'''

faces/celeba/img_align_celeba


"\n\nceleba_train_dataset = torchvision.datasets.CelebA(\n     image_path, split='train',\n     target_type='attr', download=False\n )\n"

In [ ]:
df = pd.read_csv(filepath, sep="\s+", skiprows=1, usecols=['Male'])
df.loc[df['Male'] == -1, 'Male'] = 0
df.head()

,Male
000001.jpg,0
000002.jpg,0
000003.jpg,1
000004.jpg,0
000005.jpg,0


In [ ]:
num_train_examples = int(df.shape[0]*0.8)
df_train = df.iloc[:num_train_examples, :]
df_test = df.iloc[num_train_examples:, :]
print('Number of male and female images in training dataset:')
np.bincount(df_train['Male'].values)

Number of male and female images in training dataset:


array([94093, 67986])

In [ ]:
print('Number of male and female images in test dataset:')
np.bincount(df_test['Male'].values)

Number of male and female images in test dataset:


array([24072, 16448])

In [ ]:
class CelebaDataset(Dataset):
    """Custom Dataset for loading CelebA face images"""

    def __init__(self, txt_path, img_dir, transform=None):
    
        df = pd.read_csv(filepath, sep="\s+", skiprows=1, usecols=['Male'])
        self.img_dir = img_dir
        self.txt_path = txt_path
        self.img_names = df.index.values
        self.y = df['Male'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        return img, label

    def __len__(self):
        return self.y.shape[0]

In [ ]:
transform_train = transforms.Compose([
     transforms.RandomCrop([178, 178]),
     transforms.RandomHorizontalFlip(),
     transforms.Resize([224, 224]),
     transforms.ToTensor(),
 ])
transform = transforms.Compose([
     transforms.CenterCrop([178, 178]),
     transforms.Resize([224, 224]),
     transforms.ToTensor(),
 ])

custom_transform = transforms.Compose([transforms.Grayscale(),                                       
                                       #transforms.Lambda(lambda x: x/255.),
                                       transforms.ToTensor()])

dataset = CelebaDataset(txt_path=filepath,
                              img_dir=dataset_folder,
                              transform=transform_train)
celeba_train_dataset,celeba_valid_dataset,celeba_test_dataset= torch.utils.data.random_split(dataset, [0.8,0.1,0.1])

celeba_train_loader = DataLoader(dataset=celeba_train_dataset,
                          batch_size=8,
                          shuffle=True,
                          num_workers=4) 


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
'''
celeba_valid_dataset = torchvision.datasets.CelebA(
     image_path, split='valid',
     target_type='attr', download=True
 )

celeba_test_dataset = torchvision.datasets.CelebA(
     image_path, split='test',
     target_type='attr', download=True
 )
'''

"\nceleba_valid_dataset = torchvision.datasets.CelebA(\n     image_path, split='valid',\n     target_type='attr', download=True\n )\n\nceleba_test_dataset = torchvision.datasets.CelebA(\n     image_path, split='test',\n     target_type='attr', download=True\n )\n"

In [ ]:
#print('Train set:', len(celeba_train_dataset))
#print('Validation set:', len(celeba_valid_dataset))
#print('Test set:', len(celeba_test_dataset))

In [ ]:
#! ls faces/celeba/img_align_celeba

# Smile Calssification Dataloader

In [ ]:
'''
from torch.utils.data import DataLoader
get_smile = lambda attr: attr[31]
v=get_smile
print(type(v))
transform_train = transforms.Compose([
     transforms.RandomCrop([178, 178]),
     transforms.RandomHorizontalFlip(),
     transforms.Resize([224, 224]),
     transforms.ToTensor(),
 ])
transform = transforms.Compose([
     transforms.CenterCrop([178, 178]),
     transforms.Resize([224, 224]),
     transforms.ToTensor(),
 ])

celeba_train_dataset_smile = torchvision.datasets.CelebA(
     image_path, split='train',
     target_type='attr', download=False,
     transform=transform_train#, target_transform=get_smile
 )
torch.manual_seed(1)
data_loader_smile = DataLoader(celeba_train_dataset_smile, batch_size=16)
celeba_valid_dataset_smile = torchvision.datasets.CelebA(
     image_path, split='valid',
     target_type='attr', download=False,
     transform=transform#, target_transform=get_smile
 )
celeba_test_dataset_smile = torchvision.datasets.CelebA(
     image_path, split='test',
     target_type='attr', download=False,
     transform=transform#, target_transform=get_smile
 )
#Training our model on 16000 dataponints initially  and 1000 validation poitns
#Change this later
from torch.utils.data import Subset
celeba_train_dataset = Subset(celeba_train_dataset_smile,
                               torch.arange(50000))
celeba_valid_dataset = Subset(celeba_valid_dataset_smile,
                               torch.arange(5000))
print('Train set:', len(celeba_train_dataset))

print('Validation set:', len(celeba_valid_dataset))
'''


"\nfrom torch.utils.data import DataLoader\nget_smile = lambda attr: attr[31]\nv=get_smile\nprint(type(v))\ntransform_train = transforms.Compose([\n     transforms.RandomCrop([178, 178]),\n     transforms.RandomHorizontalFlip(),\n     transforms.Resize([224, 224]),\n     transforms.ToTensor(),\n ])\ntransform = transforms.Compose([\n     transforms.CenterCrop([178, 178]),\n     transforms.Resize([224, 224]),\n     transforms.ToTensor(),\n ])\n\nceleba_train_dataset_smile = torchvision.datasets.CelebA(\n     image_path, split='train',\n     target_type='attr', download=False,\n     transform=transform_train#, target_transform=get_smile\n )\ntorch.manual_seed(1)\ndata_loader_smile = DataLoader(celeba_train_dataset_smile, batch_size=16)\nceleba_valid_dataset_smile = torchvision.datasets.CelebA(\n     image_path, split='valid',\n     target_type='attr', download=False,\n     transform=transform#, target_transform=get_smile\n )\nceleba_test_dataset_smile = torchvision.datasets.CelebA(\n    

In [ ]:
'''
from torch.utils.data import DataLoader
get_gender = lambda attr: attr[21]
#a =get_gender

celeba_train_dataset_gender = torchvision.datasets.CelebA(
     image_path, split='train',
     target_type='attr', download=False,
     transform=transform_train#, target_transform=get_gender
 )
torch.manual_seed(1)
data_loader_smile = DataLoader(celeba_train_dataset_gender, batch_size=16)
celeba_valid_dataset_gender = torchvision.datasets.CelebA(
     image_path, split='valid',
     target_type='attr', download=False,
     transform=transform#, target_transform=get_gender
 )
celeba_test_dataset_gender = torchvision.datasets.CelebA(
     image_path, split='test',
     target_type='attr', download=False,
     transform=transform#, target_transform=get_gender
 )
'''

"\nfrom torch.utils.data import DataLoader\nget_gender = lambda attr: attr[21]\n#a =get_gender\n\nceleba_train_dataset_gender = torchvision.datasets.CelebA(\n     image_path, split='train',\n     target_type='attr', download=False,\n     transform=transform_train#, target_transform=get_gender\n )\ntorch.manual_seed(1)\ndata_loader_smile = DataLoader(celeba_train_dataset_gender, batch_size=16)\nceleba_valid_dataset_gender = torchvision.datasets.CelebA(\n     image_path, split='valid',\n     target_type='attr', download=False,\n     transform=transform#, target_transform=get_gender\n )\nceleba_test_dataset_gender = torchvision.datasets.CelebA(\n     image_path, split='test',\n     target_type='attr', download=False,\n     transform=transform#, target_transform=get_gender\n )\n"

In [ ]:
def comp_model(model1,model2):
  for p1, p2 in zip(model1.parameters(), model2.parameters()):
      if p1.data.ne(p2.data).sum() > 0:
          return False
  return True
#print("Comparing models",comp_model(model_gender,model_smile))

In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')


In [ ]:
def validate_state_dicts(model_state_dict_1, model_state_dict_2):
    if len(model_state_dict_1) != len(model_state_dict_2):
        logger.info(
            f"Length mismatch: {len(model_state_dict_1)}, {len(model_state_dict_2)}"
        )
        return False

    # Replicate modules have "module" attached to their keys, so strip these off when comparing to local model.
    if next(iter(model_state_dict_1.keys())).startswith("module"):
        model_state_dict_1 = {
            k[len("module") + 1 :]: v for k, v in model_state_dict_1.items()
        }

    if next(iter(model_state_dict_2.keys())).startswith("module"):
        model_state_dict_2 = {
            k[len("module") + 1 :]: v for k, v in model_state_dict_2.items()
        }

    for ((k_1, v_1), (k_2, v_2)) in zip(
        model_state_dict_1.items(), model_state_dict_2.items()
    ):
        if k_1 != k_2:
            logger.info(f"Key mismatch: {k_1} vs {k_2}")
            return False
        # convert both to the same CUDA device
        if str(v_1.device) != "cuda:0":
            v_1 = v_1.to("cuda:0" if torch.cuda.is_available() else "cpu")
        if str(v_2.device) != "cuda:0":
            v_2 = v_2.to("cuda:0" if torch.cuda.is_available() else "cpu")

        if not torch.allclose(v_1, v_2):
            logger.info(f"Tensor mismatch: {v_1} vs {v_2}")
            return False

In [ ]:
'''
batch_size = 32
torch.manual_seed(1)
gender_train_dl = DataLoader(celeba_train_dataset_gender,
                       batch_size, shuffle=True)
gender_valid_dl = DataLoader(celeba_valid_dataset_gender,
                       batch_size, shuffle=False)
gender_test_dl = DataLoader(celeba_test_dataset_gender,
                     batch_size, shuffle=False)
                     '''

'\nbatch_size = 32\ntorch.manual_seed(1)\ngender_train_dl = DataLoader(celeba_train_dataset_gender,\n                       batch_size, shuffle=True)\ngender_valid_dl = DataLoader(celeba_valid_dataset_gender,\n                       batch_size, shuffle=False)\ngender_test_dl = DataLoader(celeba_test_dataset_gender,\n                     batch_size, shuffle=False)\n                     '

In [ ]:

from torch.utils.data import DataLoader
get_smile = lambda attr: attr[31]
smile_attr_idx=31
gender_attr_idx=20
'''
celeba_train_dataset = torchvision.datasets.CelebA(
     image_path, split='train',
     target_type='attr', download=False,
     transform=transform_train
 )


celeba_valid_dataset = torchvision.datasets.CelebA(
     image_path, split='valid',
     target_type='attr', download=False,
     transform=transform
 )
celeba_test_dataset = torchvision.datasets.CelebA(
     image_path, split='test',
     target_type='attr', download=False,
     transform=transform
 )
#Training our model on 16000 dataponints initially  and 1000 validation poitns
#Change this later
from torch.utils.data import Subset
celeba_train_dataset = Subset(celeba_train_dataset,
                               torch.arange(50000))
celeba_valid_dataset = Subset(celeba_valid_dataset,
                               torch.arange(5000))
print('Train set:', len(celeba_train_dataset))

print('Validation set:', len(celeba_valid_dataset))
'''

"\nceleba_train_dataset = torchvision.datasets.CelebA(\n     image_path, split='train',\n     target_type='attr', download=False,\n     transform=transform_train\n )\n\n\nceleba_valid_dataset = torchvision.datasets.CelebA(\n     image_path, split='valid',\n     target_type='attr', download=False,\n     transform=transform\n )\nceleba_test_dataset = torchvision.datasets.CelebA(\n     image_path, split='test',\n     target_type='attr', download=False,\n     transform=transform\n )\n#Training our model on 16000 dataponints initially  and 1000 validation poitns\n#Change this later\nfrom torch.utils.data import Subset\nceleba_train_dataset = Subset(celeba_train_dataset,\n                               torch.arange(50000))\nceleba_valid_dataset = Subset(celeba_valid_dataset,\n                               torch.arange(5000))\nprint('Train set:', len(celeba_train_dataset))\n\nprint('Validation set:', len(celeba_valid_dataset))\n"

In [ ]:
batch_size = 16
torch.manual_seed(1)
train_dl = DataLoader(celeba_train_dataset,
                       batch_size, shuffle=True)
valid_dl = DataLoader(celeba_valid_dataset,
                       batch_size, shuffle=False)
test_dl = DataLoader(celeba_test_dataset,
                     batch_size, shuffle=False)

In [ ]:
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2
feature_extract=True
num_epochs=5

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        model_ft.fc=nn.Sequential(
                model_ft.fc,
                nn.Softmax(),
                )
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)

        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained='vggface2')
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
##model_smile, input_size = initialize_model(model_name, num_classes, feature_extract=True, use_pretrained=True)
model_gender,_= initialize_model(model_name, num_classes, feature_extract=True, use_pretrained=True)
#print(type(model_smile))
#model_smile = model_smile.to(device)
#model_gender =model_gender.to(device)
# Print the model we just instantiated
#compare_models(model_gender,model_smile)
#print(validate_state_dicts(model_gender.state_dict(),model_smile.state_dict()))

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(model_gender)
model_gender=model_gender.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Send the model to GPU


# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
'''

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
'''
# Observe that all parameters are being optimized
def return_optimizer(model):
    return  optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
dataloaders_dict ={'train':train_dl,'val':valid_dl}
#gender_dataloader_dict={'train':gender_train_dl,'val':gender_valid_dl}

In [ ]:

def train_model(model, dataloaders, criterion, optimizer, type,num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels= labels.to(device)
                #print( "Smile ",labels[:,smile_attr_idx])
                #print("Gender",labels[:,gender_attr_idx])
                '''
                if (type=='smile'):

                 labels = labels[:,smile_attr_idx].squeeze().to(device)
                if (type=='gender'):

                 labels = labels[:,gender_attr_idx].squeeze().to(device)
                '''
                print("Label",(labels))

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        print(outputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
'''

smile_optimizer=return_optimizer(model_smile)
# Train and evaluate
model_smile, hist = train_model(model_smile, dataloaders_dict, criterion, smile_optimizer,'smile', num_epochs=num_epochs, is_inception=False)
'''

"\n\nsmile_optimizer=return_optimizer(model_smile)\n# Train and evaluate\nmodel_smile, hist = train_model(model_smile, dataloaders_dict, criterion, smile_optimizer,'smile', num_epochs=num_epochs, is_inception=False)\n"

In [ ]:
gender_optimizer=return_optimizer(model_gender)
# Train and evaluate
model_ft, hist = train_model(model_gender, dataloaders_dict, criterion, gender_optimizer,'gender', num_epochs=num_epochs, is_inception=False)

Epoch 0/4
----------
Label tensor([-1,  1, -1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1],
       device='cuda:0')


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.5646, 0.4354],
        [0.5256, 0.4744],
        [0.6759, 0.3241],
        [0.7457, 0.2543],
        [0.6889, 0.3111],
        [0.6088, 0.3912],
        [0.6450, 0.3550],
        [0.6208, 0.3792],
        [0.7197, 0.2803],
        [0.6849, 0.3151],
        [0.8442, 0.1558],
        [0.5369, 0.4631],
        [0.6109, 0.3891],
        [0.5874, 0.4126],
        [0.6406, 0.3594],
        [0.6823, 0.3177]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


RuntimeError: ignored

In [ ]:
compare_models(model_smile,model_gender)

In [ ]:
smile_model_name="smile_model3.pt"
save_path="/content/gdrive/MyDrive/FairFaceRecExptResults/"+smile_model_name
torch.save(model_smile.state_dict(),save_path )

In [ ]:
#gender_model=copy.deepcopy(model_ft)



gender_model_name="gender_model3.pt"
save_path="/content/gdrive/MyDrive/FairFaceRecExptResults/"+gender_model_name
torch.save(model_gender.state_dict(),save_path )



In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')
compare_models(gender_model,model_ft)

In [ ]:

#model = model_ft
#model = model.to(device)
#model_weights = torch.load(save_path,map_location=device)
#model.load_state_dict(model_weights)
#model.eval()

In [ ]:
# deep Copying a model 
model_gender=copy.deepcopy(model)

In [ ]:
# Do this after adding the new layer
#model_ft, hist = train_model(model_gender, dataloaders_dict, criterion, optimizer_ft,'gender', num_epochs=num_epochs, is_inception=(model_name=="densenet"))

In [ ]:
labels=["smiling","not smiling"]
def predict(input_img,model=model):
    
    input=transform(input_img)
    input = input.unsqueeze(0)
    input = input.to(device)
    model= model.to(device)
    
    with torch.no_grad():
      prediction = torch.nn.functional.softmax(model(input), dim=1)
      output=model(input)
      print("output",prediction,model(input))
      #print(" Predcition only ",)
      predcition=prediction.cpu().numpy()
      print(prediction)
      confidences= {labels[i]:float(prediction[i]) for i in range (0,len(labels))}

    #idx2class={v: k for v, k in enumerate(dataframe.columns[1:].values)}
    #confidences = {idx2class[i]: float(prediction[i]) for i in idx2class.values()}  
    return  confidences

In [ ]:
class new_model (nn.Module):
  def __init__(self, output_layer,pretrained_model,num_classes=2,feature_extract=True):
    super().__init__()
    self.pretrained_model=pretrained_model
    self.output_layer=output_layer
    self.layers=list(self.pretrained_model._modules.keys())
    self.layer_count=0
    self.feature_extract=feature_extract
    self.num_classes=num_classes
    set_parameter_requires_grad(pretrained_model,self.feature_extract)

    for l in self.layers:
          print(self.layers)
          if l != self.output_layer:
                self.layer_count += 1
          else:
                break
    for i in range(1,len(self.layers)-self.layer_count):
          self.dummy_var = self.pretrained_model._modules.pop(self.layers[-i])
    #last_layer_shape=self.layers[-1].shape 
    #print(last_layer_shape)
    #num_features=1
    #for i in range(1,len(last_layer_shape)):
      #num_features=num_features*last_layer_shape[i]
    #self.num_features=self.layers[-1].shape[1]*self.layers
    self.classification_layer=nn.Sequential(torch.nn.LazyLinear(num_classes))
    self.net = nn.Sequential(self.pretrained_model._modules)
    self.pretrained = None

  def forward(self,x):
    x = self.net(x)
    #print(x.shape)
    batch_size=x.shape[0]

    x=torch.flatten(x,start_dim=1)
    #print(x.shape)
    num_features=x.shape[1]
    #classification_layer=nn.Linear(num_features,self.num_classes).to(device)
    #print(classification_layer)
    #x=classification_layer(nn.Sex)
    #x=torch.nn.functional.softmax(x)
    #print(x.shape)
    x=self.classification_layer(x)
    #print(x.shape)
    return (x)
# Test case


In [ ]:
gender_model=new_model('avgpool',model_gender)
gender_model.to(device)

In [ ]:
gender_model.eval()

In [ ]:
gender_model2,_=initialize_model(model_name, num_classes, feature_extract=True, use_pretrained=True)

In [ ]:
params_to_update = gender_model2.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in gender_model2.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in gender_model2.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

In [ ]:
criterion = nn.CrossEntropyLoss()
gender_model2.to(device)
gender_model2, hist = train_model(gender_model2, gender_dataloader_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="densenet"))

In [ ]:
gender_model_name="gender_model_real.pt"
save_path="/content/gdrive/MyDrive/FairFaceRecExptResults/"+gender_model_name
torch.save(gender_model.state_dict(),save_path )

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr

demo=gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=gr.Label())
demo.launch(debug=True)